In [1]:
import pybel
import os
import logging
import nx2d3
import py2neo
import json
import time

In [2]:
logging.basicConfig(level=logging.INFO)
logging.getLogger("requests").setLevel(logging.WARNING)
logging.getLogger('neo4j.bolt').setLevel(logging.WARNING)

In [3]:
time.asctime()

'Mon Oct 10 00:59:08 2016'

In [4]:
base = os.environ['PYBEL_BASE']
path = os.path.join(base, 'tests', 'bel', 'test_bel_1.bel')

In [5]:
g = pybel.from_path(path)

INFO:pybel.parser.utils:File length: 26 lines
INFO:pybel.graph:Finished parsing document section in 0.01 seconds
INFO:pybel.graph:Finished parsing definitions section in 2.04 seconds
INFO:pybel.graph:Finished parsing statements section in 43.84 seconds


In [6]:
nx2d3.embed_networkx(g, height=300, width=600)

<IPython.core.display.Javascript object>

In [7]:
print_edges = True
if print_edges:
    for u,v,data in g.edges(data=True):
        print(u, v, json.dumps(data, indent=2))

('Gene', 'ALPHABET', 'E') ('RNA', 'ALPHABET', 'E') {
  "relation": "transcribedTo"
}
('RNA', 'ALPHABET', 'B') ('Protein', 'ALPHABET', 'B') {
  "relation": "translatedTo"
}
('Protein', 'ALPHABET', 'A') ('Protein', 'ALPHABET', 'B') {
  "citation_type": "Pubmed",
  "citation_reference": "123455",
  "Evidence": "Evidence 1 w extra notes",
  "citation_name": "That one article from last week",
  "relation": "increases"
}
('Protein', 'ALPHABET', 'D') ('Protein', 'ALPHABET', 'E') {
  "object": {
    "modifier": "Translocation",
    "effect": {
      "fromLoc": {
        "namespace": "GOCC",
        "name": "intracellular"
      },
      "toLoc": {
        "namespace": "GOCC",
        "name": "cell surface"
      }
    }
  },
  "subject": {
    "modifier": "Activity",
    "effect": {
      "MolecularActivity": "KinaseActivity"
    }
  },
  "citation_type": "Pubmed",
  "citation_reference": "123456",
  "relation": "association",
  "TESTAN1": "2",
  "Evidence": "Evidence 3",
  "citation_name": "T

In [8]:
neo_path = os.environ['NEO_PATH']
neo = py2neo.Graph(neo_path)
neo.data('match (n) detach delete n')

INFO:httpstream:> GET http://localhost:7474/db/data/
INFO:httpstream:< 200 OK [795]


[]

In [9]:
g.to_neo4j(neo)

In [10]:
neo.data('match (n) return n')

[{'n': (`0`:Gene {name:"0",namespace:"ALPHABET",value:"E"})},
 {'n': (`13`:Protein {name:"13",namespace:"ALPHABET",value:"C"})},
 {'n': (`1`:RNA {name:"1",namespace:"ALPHABET",value:"B"})},
 {'n': (`2`:Protein {name:"2",namespace:"ALPHABET",value:"A"})},
 {'n': (`3`:Protein {name:"3",namespace:"ALPHABET",value:"D"})},
 {'n': (`4`:RNA {name:"4",namespace:"ALPHABET",value:"C"})},
 {'n': (`5`:RNA {name:"5",namespace:"ALPHABET",value:"D"})},
 {'n': (`6`:Protein {name:"6",namespace:"ALPHABET",value:"B"})},
 {'n': (`7`:Gene {name:"7",namespace:"ALPHABET",value:"C"})},
 {'n': (`8`:Protein {name:"8",namespace:"ALPHABET",value:"E"})},
 {'n': (`9`:Gene {name:"9",namespace:"ALPHABET",value:"D"})},
 {'n': (`10`:RNA {name:"10",namespace:"ALPHABET",value:"E"})},
 {'n': (`11`:Gene {name:"11",namespace:"ALPHABET",value:"A"})},
 {'n': (`12`:Gene {name:"12",namespace:"ALPHABET",value:"B"})},
 {'n': (`14`:RNA {name:"14",namespace:"ALPHABET",value:"A"})}]

In [11]:
r = neo.run('match (n)-[k]-(m) return n.namespace, n.value, k.relation, m.namespace, m.value')
r.dump()

 n.namespace  n.value  k.relation         m.namespace  m.value 
---------------------------------------------------------------
 ALPHABET     E        transcribedTo      ALPHABET     E       
 ALPHABET     C        decreases          ALPHABET     B       
 ALPHABET     C        translatedTo       ALPHABET     C       
 ALPHABET     B        transcribedTo      ALPHABET     B       
 ALPHABET     B        translatedTo       ALPHABET     B       
 ALPHABET     A        translatedTo       ALPHABET     A       
 ALPHABET     A        increases          ALPHABET     B       
 ALPHABET     D        association        ALPHABET     E       
 ALPHABET     D        translatedTo       ALPHABET     D       
 ALPHABET     D        directlyDecreases  ALPHABET     B       
 ALPHABET     D        association        ALPHABET     E       
 ALPHABET     C        transcribedTo      ALPHABET     C       
 ALPHABET     C        translatedTo       ALPHABET     C       
 ALPHABET     D        transcribedTo    